# Path Techniques #

This notebook will discuss some useful geometrical manipulations on paths:
* [Arrowheads](#Arrowheads)
* [General dash shapes](#General-Dash-Shapes)
* [Filleting of corners](#Filleting-Of-Corners)

First, the usual common definitions for rendering of illustrations...

In [ ]:
from ipywidgets.widgets import interact
from IPython.display import display_png
import math
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Path, \
    Rect, \
    Vector
Pt = Path.Point

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (400, 400)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
       .save()
       .set_source_colour(Colour.grey(.95))
       .paint()
       .restore()
       .select_font_face(family = "serif", slant = CAIRO.FONT_SLANT_NORMAL, weight = CAIRO.FONT_WEIGHT_NORMAL)
       .set_font_size(12)
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

reset()

## Arrowheads ##

Supposing you want to draw an arrow symbol. Cairo paths can be curves, not just straight lines, and it lets you specify round, butt or square end caps for the ends of the stroke, but what if you wanted to put some other shape there? How would you put an arrow head and a tail on an arbitrary Cairo path? Clearly, these have to be drawn as separate shapes. But you want to correctly rotate them to line up with the direction of the path at its corresponding ends.

Here is an example path that we will use for our arrow:

In [ ]:
margin = Vector(40, 40)

sample_path = \
    Path \
      ((
        Path.Segment
          (
            points =
              (
                Pt((0, 0), False),
                Pt((1, 0.2), True),
                Pt((1, 0.3), True),
                Pt((0.5, 0.5), False),
                Pt((0, 0.7), True),
                Pt((0, 0.8), True),
                Pt((1, 1), False),
              ),
            closed = False
          ),
      )).transform(Matrix.translate(margin) * Matrix.scale(pix.dimensions - 2 * margin))

def draw_sample_path() :
    (ctx
        .save()
        .new_path()
        .append_path(sample_path)
        .set_line_width(10)
        .stroke()
        .restore()
    )
#end draw_sample_path
reset()
ctx.set_source_colour(Colour.grey(0))
draw_sample_path()
display()

And here are the shapes we will use for our arrow head and arrow tail. Note their initial unrotated orientation is pointing to the right, because that corresponds to the 0° orientation in Cairo:

In [ ]:
arrow_head = \
    Path \
      ((
        Path.Segment
          (
            points =
              (
                Pt((-20, -20), False),
                Pt((+20, 0), False),
                Pt((-20, +20), False),
              ),
            closed = True
          ),
      ))

arrow_tail = \
    Path \
      ((
        Path.Segment
          (
            points =
              (
                Pt((-10, -10), False),
                Pt((+10, 0), False),
                Pt((-10, +10), False),
              ),
            closed = False
          ),
      ))

reset()
(ctx
    .set_line_width(10)
    .new_path()
    .append_path(arrow_head, Matrix.translate((50, 50)))
    .fill()
    .append_path(arrow_tail, Matrix.translate((50, 100)))
    .stroke()
)
display()

To compute the appropriate orientation at each end of a path, let’s try something really simple: look at the control point at the end of the path, and the one next to it, and use the direction between these two to orient the arrow head or tail:

In [ ]:
def draw_arrow_for_path(ctx, line_path, arrow_head, fill_arrow_head, arrow_tail, fill_arrow_tail) :
    # assuming line_path has been stroked into ctx, puts arrow head and tail and/or on it.
    # Each shape can be filled or stroked.
    if arrow_head != None :
        # arrow head goes at end of path
        p0 = line_path.segments[-1].points[-1].pt
        p1 = line_path.segments[-1].points[-2].pt
        (ctx
            .new_path()
            .append_path
             (
                arrow_head,
                    Matrix.translate(p0)
                *
                    Matrix.rotate((p0 - p1).angle())
                      # note direction: arrow head points out of path
             )
        )
        if fill_arrow_head :
            ctx.fill()
        else :
            ctx.stroke()
        #end if
    #end if
    if arrow_tail != None :
        # arrow tail goes at start of path
        p0 = line_path.segments[0].points[0].pt
        p1 = line_path.segments[0].points[1].pt
        (ctx
            .new_path()
            .append_path
              (
                arrow_tail,
                    Matrix.translate(p0)
                *
                    Matrix.rotate((p1 - p0).angle())
                      # note direction: arrow tail points into path
              )
        )
        if fill_arrow_tail :
            ctx.fill()
        else :
            ctx.stroke()
        #end if
    #end if
#end draw_arrow_for_path

Here is what the result looks like with our example paths:

In [ ]:
reset()
ctx.set_source_colour(Colour.grey(0))
ctx.set_line_width(10)
draw_sample_path()
draw_arrow_for_path(ctx, sample_path, arrow_head, True, arrow_tail, False)
display()

## General Dash Shapes ##

Cairo allows you to stroke a path using a dash pattern. But what if you want to distribute instances of some more general shape along the path?

To do this, you need to inspect the entire geometry of the path. To simplify things when the path contains a curve, you can ask Cairo to flatten the path, so you only have to deal with straight-line segments.

The basic technique for positioning a custom dash shape along such a path are:
* Flatten the path to straight-line segments.
* Sum up the length of each segment in turn. When we have accumulated a length at least equal to the spacing between positions of the dash shape, then draw another dash shape, oriented the same way as the last path segment. (Or—exercise for the reader—we could look at the average orientation of the path segments contributing to this length.)

The following code adjusts the specified dash spacing so it evenly divides the curve length. This requires doing two passes over the flattened curve, the first to compute the total curve length.

In [ ]:
def draw_shape_dashes(ctx, path_to_dash, dash_path, spacing) :

    def draw_dash(pos, angle) :
        (ctx
            .new_path()
            .append_path(dash_path, Matrix.translate(pos) * Matrix.rotate(angle))
            .stroke()
        )
    #end draw_dash

#begin draw_shape_dashes
    for seg in path_to_dash.flatten(tolerance = ctx.tolerance).segments :
        total_length = 0
        for piece in seg.pieces() :
            #print("piece = {}".format(piece)) # debug
            assert len(piece) == 2
            total_length += abs(piece[1] - piece[0])
        #end for
        use_spacing = total_length / round(total_length / spacing)
          # adjusted to evenly divide total length of curve
          # fixme: possible ZeroDivisionError if total_length < spacing
        prev_piece = None
        rem_length = 0
        for piece in seg.pieces() :
            if prev_piece == None :
                if not seg.closed :
                    draw_dash(piece[0], (piece[1] - piece[0]).angle())
                #end if
            piece_length = abs(piece[1] - piece[0])
            dash_offset = use_spacing - rem_length
            rem_length += piece_length
            while rem_length >= use_spacing :
                draw_dash \
                  (
                    pos =
                            (piece[1] - piece[0]) * dash_offset / piece_length
                        +
                            piece[0],
                    angle = (piece[1] - piece[0]).angle()
                  )
                dash_offset += use_spacing
                rem_length -= use_spacing
            #end while
            prev_piece = piece
        #end for
        if prev_piece != None and not seg.closed and rem_length >= use_spacing / 2 :
            # check on rem_length to prevent rounding errors from causing this
            # last one to be done twice
            draw_dash(prev_piece[1], (prev_piece[1] - prev_piece[0]).angle())
        #end if
    #end for
#end draw_shape_dashes

Here is an example use of the `draw_shape_dashes` routine: `dash_path` defines the dash shape to distribute along the `sample_path`.

In [ ]:
dash_path = \
    Path \
      ((
        Path.Segment
          (
            points =
              (
                  Pt((-0.5, -0.5), False),
                  Pt((0.5, 0), False),
                  Pt((-0.5, 0.5), False),
              ),
            closed = False
          ),
      ))

reset()
ctx.set_source_colour(Colour.from_hsva((0.55, 0.5, 0.8)))
draw_sample_path()
ctx.set_source_colour(Colour.grey(0)).set_line_width(2)
draw_shape_dashes \
  (
    ctx = ctx,
    path_to_dash = sample_path,
    dash_path = dash_path.transform(Matrix.scale(5)),
    spacing = 15
  )
display()

## Filleting Of Corners ##

A *fillet* is the rounding off of a corner, replacing it with a circular arc. So given two straight lines $P_0P_1$ and $P_1P_2$ between the points $P_0$, $P_1$ and $P_2$, how can we compute an arc of radius $r$ that will replace the corner $P_1$ and smoothly join onto the two straight lines?

Before we consider the general case, let us look at a specific diagram:

In [ ]:
def fillet_diagram_1() :
    # draws a simple diagram to illustrate the fillet problem.
    # The code that actually solves the general problem will come below.
    reset()
    p1 = Vector(350, 200)
    linevec = Vector(250, 0)
    p0 = p1 + linevec.rotate(-135 * qah.deg)
    p2 = p1 + linevec.rotate(135 * qah.deg)
    r = 100
    t1 = p1 + (p0 - p1).norm() * r
    t2 = p1 + (p2 - p1).norm() * r
    centre = p1 - Vector(r, 0) * math.sqrt(2)
    ctx.set_font_size(24)
    ctx.line_width = 5
    (ctx
        .move_to(p0)
        .line_to(p1)
        .line_to(p2)
        .new_sub_path()
        .arc
          (
            centre = centre,
            radius = r,
            angle1 = -45 * qah.deg,
            angle2 = 45 * qah.deg,
            negative = False
          )
        .stroke()
    )
    ctx.line_width = 2.5
    (ctx
        .move_to(centre)
        .line_to(t1)
        .move_to(centre)
        .line_to(t2)
        .stroke()
        .move_to((centre + t1) / 2 + Vector(8, 8))
        .show_text("r")
    )
    for pos, label in \
        (
            (p0, "P0"),
            (p1, "P1"),
            (p2, "P2"),
            (centre, "C"),
            (t1, "T1"),
            (t2, "T2"),
        ) \
    :
        (ctx
            .new_sub_path()
            .arc
              (
                centre = pos,
                radius = 5,
                angle1 = 0,
                angle2 = qah.circle,
                negative = False
              )
            .fill()
            .move_to(pos + Vector(10, 10))
            .show_text(label)
        )
    #end for
    display()
#end fillet_diagram_1
fillet_diagram_1()

The above shows the points $P_0$, $P_1$ and $P_2$ as previously described. It also shows the additional calculated points $C$, the centre of the fillet circle, and $T_1$ and $T_2$, being the tangent points between the circle and the straight lines $P_0P_1$ and $P_1P_2$ respectively. (Note the line lengths $CT_1 = CT_2 = r$, since the tangent points lie on the circle.) So the line segments $T_1P_1$ and $T_2P_2$ are deleted (“trimmed”, in CAD/graphics parlance), and the resulting shape consists of the remaining straight line segments $P_0T_1$ and $T_2P_2$ plus the arc $T_1T_2$.

Consider the triangles $CT_1P_1$ and $CT_2P_1$: because $CT_1 = CT_2$, they share a common side $CP_1$, and the corresponding angles $CT_1P_1 = CT_2P_1 = 90°$, the triangles must be equal. (The equal angle is not the included angle between the equal sides, but that’s OK because it is 90° or greater.) Therefore the line lengths $P_1T_1 = P_1T_2$.

Now let us take a closer look at one of these triangles.

In [ ]:
def fillet_diagram_2() :
    # another diagram to illustrate the fillet problem.
    reset()
    p1 = Vector(350, 200)
    r = 250
    theta = 120 * qah.deg # arbitrary, just for example
    centre = p1 - Vector(1, 0) * r / math.sin(theta / 2)
    t1 = centre + (p1 - centre).rotate(- 90 * qah.deg + theta / 2) * math.sin(theta / 2)
    ctx.set_font_size(24)
    ctx.line_width = 5
    (ctx
        .move_to(p1)
        .line_to(t1)
        .line_to(centre)
        .close_path()
        .stroke()
    )
    for pos, label in \
        (
            (p1, "P1"),
            (centre, "C"),
            (t1, "T1"),
        ) \
    :
        (ctx
            .new_sub_path()
            .arc
              (
                centre = pos,
                radius = 5,
                angle1 = 0,
                angle2 = qah.circle,
                negative = False
              )
            .fill()
            .move_to(pos + Vector(10, 20))
            .show_text(label)
        )
    #end for
    ctx.line_width = 2.5
    angle_radius = 25
    ctx.set_font_size(18)
    for label, side, pr, pmid, pl in \
        (
            ("\u03f4", False, t1, p1, centre),
            ("\u03c0/2 - \u03f4", True, p1, centre, t1),
        ) \
    :
        angle1 = (pl - pmid).angle()
        angle2 = (pr - pmid).angle()
        (ctx
            .move_to(pmid + Vector.unit(angle1) * angle_radius)
            .arc
              (
                centre = pmid,
                radius = angle_radius,
                angle1 = angle1,
                angle2 = angle2,
                negative = False
              )
            .stroke()
            .move_to(pmid + Vector(40 * (-1, 1)[side], - 10))
            .show_text(label)
        )
    #end for
    pl = t1 + Vector.unit((p1 - t1).angle()) * angle_radius
    pr = t1 + Vector.unit((centre - t1).angle()) * angle_radius
    (ctx
        .move_to(pl)
        .line_to(pl + pr - t1)
        .line_to(pr)
        .stroke()
    )
    for label, pl, pr in \
        (
            ("r", centre, t1),
            ("h", centre, p1),
            ("s", t1, p1),
        ) \
    :
        (ctx
            .move_to((pl + pr) / 2 + Vector(10, 15))
            .show_text(label)
        )
    #end for
    display()
#end fillet_diagram_2
fillet_diagram_2()

Let the angle at the corner $CP_1T_1$ be $\theta$. Then, because $CT_1P_1$ is a right angle, the angle at the corner $T_1CP_1$ will equal ${\pi \over 2} - \theta$. But because $T_1$ is on the line $P_0P_1$, and similarly $T_2$ is on the line $P_2P_1$, it follows that the angle at the corner $P_0P_1P_2$ is $2\theta$, and the one subtended by $T_1CT_2$ is $\pi - 2 \theta$.

We already have $CT_1 = r$. If we let $CP_1 = h$ and $T_1P_1 = s$, then

$${r \over h} = \sin \theta$$

and

$${r \over s} = \tan \theta$$

from which

$$h = {r \over \sin \theta}$$

and

$$s = {r \over \tan \theta}$$

In Python terms, we can compute the direction of the line $P_1P_0$ as `(p0 - p1).angle()`, where `p0` and `p1` are `Vector` values representing their respective coordinates, and similarly the direction of $P_1P_2$ as `(p2 - p1).angle()`. The direction of the line $CP_1$ is halfway between these two, but we have to be careful, because there are *two* halfway directions, so we have to make sure we choose the one that goes *between* the lines in the correct direction:

    dirn1 = (p0 - p1).angle()
    dirn2 = (p2 - p1).angle()
    mid_dirn = (dirn1 + dirn2) / 2
    flip = abs(dirn2 - dirn1) > math.pi # will be used again later
    if flip :
        if mid_dirn > 0 :
            mid_dirn -= math.pi
        else :
            mid_dirn += math.pi
        #end if
    #end if

If we get the outside bisector (which we can tell because the absolute different between the line directions is greater than $\pi$), then the inside bisector is simply $±\pi$ away, *i.e.* in the opposite direction. We add or subtract $\pi$ as appropriate to keep the result within $[-\pi, \pi]$ (the same as the result of the `math.atan` function).

The distance from $P_1$ to $T_1$ (and also to $T_2$) (this is $s$ in the above formulas) is computed thus:

    inset = abs(r / math.tan((dirn2 - dirn1) / 2))

Here it doesn’t matter whether `(dirn2 - dirn1) / 2` gives us half of the inner or outer angle, since the magnitudes of the tangents are the same.

Now we can compute the positions of the points $T_1$ and $T_2$:

    t1 = p1 + Vector.unit((p0 - p1).angle()) * inset
    t2 = p1 + Vector.unit((p2 - p1).angle()) * inset

And the coordinates of the centre $C$ of the fillet circle; remember this is $h = {r \over \sin \theta}$ away from $P_1$, and its direction is given by the angle `mid_dirn`:

    fillet_centre = \
        (
            p1
        +
            r / abs(math.sin(mid_dirn - dirn1)) * Vector.unit(mid_dirn)
        )

As with the tangent formulas, it doesn’t matter whether we consider $\theta$ to be half the inner angle between the lines or half the outer angle, since the two possibilities add up to $\pi$, therefore they have the same sine.

So now the code for actually constructing the filleted path should be becoming clear. It should look something like this (where `g` is a `Context`):

    g.move_to(p0)
    g.line_to(t1)
    g.arc \
      (
        centre = fillet_centre,
        radius = r,
        angle1 = (t1 - fillet_centre).angle(),
        angle2 = (t2 - fillet_centre).angle(),
        negative = ???
      )
    g.line_to(t2)
    g.line_to(p2)

The only thing left to determine is: what should be the value passed to the `negative` argument to the `arc` call? Which way round should the arc go?

The answer is, it depends on the relative order of `dirn1` and `dirn2`, and it goes round the opposite way if the difference between these is greater than $\pi$. In other words, whether the arc goes clockwise or not is determined by

    clockwise = (dirn2 > dirn1) == flip

So now we can write out the complete `make_fillet_path` routine:

In [ ]:
def make_fillet_path(p0, p1, p2, r) :
    "given Vectors p0, p1, p2 and Real radius r, creates a Path consisting of" \
    " two connected straight lines (p0, p1) and (p1, p2), but with the corner" \
    " at p1 rounded off into a fillet of radius r."
    dirn1 = (p0 - p1).angle()
    dirn2 = (p2 - p1).angle()
    mid_dirn = (dirn1 + dirn2) / 2
    flip = abs(dirn2 - dirn1) > math.pi
    clockwise = (dirn2 > dirn1) == flip
    if flip :
        # want mid_dirn to bisect the smaller angle between dirn1 and dirn2
        if mid_dirn > 0 :
            mid_dirn -= math.pi
        else :
            mid_dirn += math.pi
        #end if
    #end if
    inset = abs(r / math.tan((dirn2 - dirn1) / 2))
      # from corner to where fillet circle is tangent to lines
    if inset > abs(p1 - p0) or inset > abs(p1 - p2) :
        raise ValueError("lines are too short for fillet")
    #end if
    t1 = p1 + Vector.unit((p0 - p1).angle()) * inset # fillet tangent point on line (p0, p1)
    t2 = p1 + Vector.unit((p2 - p1).angle()) * inset # fillet tangent point on line (p2, p1)
    fillet_centre = \
        (
            p1
        +
            r / abs(math.sin(mid_dirn - dirn1)) * Vector.unit(mid_dirn)
        )
      # position of centre of fillet circle
    g = qah.Context.create_for_dummy()
    g.move_to(p0)
    g.line_to(t1)
    g.arc \
      (
        centre = fillet_centre,
        radius = r,
        angle1 = (t1 - fillet_centre).angle(),
        angle2 = (t2 - fillet_centre).angle(),
        negative = not clockwise
      )
    g.line_to(t2)
    g.line_to(p2)
    return \
        g.copy_path()
#end make_fillet_path


And here’s an interactive example of the fillet routine in action:

In [ ]:
@interact \
  (
    angle = (45, 225, 10),
    radius = (20, 100, 20),
    orient = (-180, 180, 10),
  )
def fillet_demo(angle, radius, orient) :
    "angle is the corner angle, radius the fillet radius, and orient changes" \
    " the direction of the whole diagram, just to show that the calculations" \
    " work under all cases."
    reset()
    p1 = Vector(350, 200)
    line_len = 300
    p0 = p1 + line_len * Vector.unit((180 + angle / 2) * qah.deg)
    p2 = p1 + line_len * Vector.unit((180 - angle / 2) * qah.deg)
    (ctx
        .append_path
          (
              make_fillet_path(p0, p1, p2, radius)
                  .transform(Matrix.rotate(angle = orient * qah.deg, centre = pix.dimensions / 2))
          )
        .set_line_width(20)
        .stroke()
    )
    display()
#end fillet_demo